In [1]:
import pandas as pd
import numpy as np

In [36]:
# hscode 데이터를 블러오는 함수

def make_hs_official():
    
    data = pd.read_csv('HS코드(2023).csv', dtype = {'HS부호':str, '성질통합분류코드':str})
    data = data[['HS부호', '한글품목명', '영문품목명', '성질통합분류코드명']]
    hscode_10 = []
    for code in data['HS부호']:
        if len(code) == 9:
            hscode_10.append('0' + code)
        else:
            hscode_10.append(code)
    data['HSCD_10'] = hscode_10
    data['HS_LEN'] = data['HS부호'].apply(lambda x :len(x))
    
    return data

In [37]:
hscd_df = make_hs_official()

In [4]:
# mti data 불러오는 함수

def make_mti_df():
    mti_df = pd.read_csv("mti.csv", encoding = "cp949", dtype = {'MTICD':'str'})
    mti_df = mti_df[['MTICD', ' CMDLT_NAME', ' CMDLT_ENG_NAME', ' CD_LEVEL_CD']]
    mti_df.columns = ['MTICD', 'MTI_NAME_KR', 'MTI_NAME_EN','CD_LEVEL']
    
    mti_cds = []
    for i in range(len(mti_df)):
        if len(mti_df['MTICD'][i]) < mti_df['CD_LEVEL'][i]:
            mti_cds.append('0' + mti_df['MTICD'][i])
        else:
            mti_cds.append(mti_df['MTICD'][i])
    
    mti_df['MTICD'] = mti_cds
    
    return mti_df
        

In [5]:
mti_df = make_mti_df()

In [27]:
# mapping data 불러오는 함수

def code_map_data():
    map_df = pd.read_excel("2022년 MTi_HS 연계표.xlsx", engine = "openpyxl", dtype = {"HS_CD10":str, "MTI_CD6":str})
    map_df = map_df.iloc[1:, :3].dropna()
    
    return map_df

In [28]:
map_df = code_map_data()

In [57]:
# 최종함수 작성

def make_final_df():
    
    first_merge_df = hscd_df.merge(map_df, how = 'left', left_on = 'HSCD_10', right_on = 'HS_CD10')
    second_merge_df = first_merge_df.merge(mti_df, how = 'left', left_on ='MTI_CD6', right_on = 'MTICD')
    
    final_df =  second_merge_df[['HS_CD10', '한글품목명', '영문품목명', '성질통합분류코드명', 'MTI_CD6', 'MTI품목명(국문)', 'MTI_NAME_EN']]
    final_df.columns = ['HS_CD10', 'HS_NAME_KR', 'HS_NAME_EN', 'HS_CAT_NAME',  'MTI_CD6', 'MTI_NAME_KR', 'MTI_NAME_EN']
    
    return final_df
    

In [58]:
final_df = make_final_df()

In [59]:
final_df

,HS_CD10,HS_NAME_KR,HS_NAME_EN,HS_CAT_NAME,MTI_CD6,MTI_NAME_KR,MTI_NAME_EN
0,0101211000,농가 사육용,For farm breeding,(말),021130,말,horse
1,0101219000,기타,Other,(말),021190,기타가축,other livestocks
2,0101291000,경주말,Horses for racing,(말),021130,말,horse
3,0101299000,기타,Other,(말),021130,말,horse
4,0101300000,당나귀,Asses,(기타 산 동물),021190,기타가축,other livestocks
...,...,...,...,...,...,...,...
12460,9706102000,악기류,Musical instruments,(수집품 및 골동품),932000,골동품,antique
12461,9706103000,기타,Other,(수집품 및 골동품),932000,골동품,antique
12462,9706901000,도자기류,Ceramics,(수집품 및 골동품),932000,골동품,antique
12463,9706902000,악기류,Musical instruments,(수집품 및 골동품),932000,골동품,antique


In [60]:
final_df.to_csv("mapping_hs_mti_.csv", index = False)